In [ ]:
import numpy as np
import os

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
DATA_PATH = 'G:\signbot\DATA\MP_DATA'
actions = np.array(open('G:\signbot\DATA\classes.txt', 'r').read().split('\n'))
number_of_sequences = 100
every_sequence_length = 30
sequences, labels = [], []

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
for action in actions:
    for sequence in range(number_of_sequences):
        window = []
        for frame_num in range(every_sequence_length):
            try:
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # print(res)
            except:
                pass
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
# sequences[0]

In [ ]:
X = np.array(sequences)

In [ ]:
# X

In [ ]:
# X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
# y

In [ ]:
# y.shape

In [ ]:
X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(X,y, train_size=0.9)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_and_val,y_train_and_val, test_size=0.2)

In [ ]:
# X_train.shape

In [ ]:
# X_valid.shape

In [ ]:
# X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
batch_size = 128    
epochs = 1000

# log_dir = 'Logs', datetime.datetime.now().strftime("%Y.%m.%d-%H.%M") + '--batch__' + str(batch_size) + '--epochs__' + str(epochs)
# tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=epochs, shuffle=True, batch_size=batch_size, validation_data= (X_valid, y_valid))

In [ ]:
model.summary()

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

In [ ]:
model.save('G:/signbot/MODEL/action1.h5')